# CS 838 Report Stage 3 Entity Matching
* Ying Li, Fan Wu, Wei Li
* {li528,fwu49,wli284}@wisc.edu 

## Introduction
Our project aims to developing an entity matcher for resturants information in city San Francisco. Two tables are crawed from TripAdvisor and Yelp website. Each table contains about 3000 tuples for individual resturants with the  attributes including the name, address, zipcode, cuisine, rating, and phone number. 

The raw information crawled from websites is stored in txt file and then pasrsed into csv file by a python script.

### Setting file path

In [1]:
import sys
import py_entitymatching as em
import pandas as pd
import os

In [2]:
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.5.2 |Anaconda 2.4.1 (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
pandas version: 0.19.2
magellan version: 0.1.0


In [3]:
import os
os.chdir('.') # replace it with the actual code directory

### List raw rxt dataset

In [4]:
!ls raw_data/

silicon_valley_dataset          yelp_crawling_price_3.txt
tripadvisor_crawling_result.txt yelp_crawling_price_4.txt
yelp_crawling_price_1.txt       yelp_crawling_price_5.txt
yelp_crawling_price_2.txt


Raw txt file containing resturant information crawled from trip advisor

In [5]:
!head -30 raw_data/tripadvisor_crawling_result.txt

----------------------------------------------------------------
title: Don of The Bimini Twist
popularity: #1 of 4,344 Restaurants in San Francisco
rating: 5 of 5 bubbles
price: $$ - $$$
cuisine: Japanese,American,Seafood
phone: 14154951900
food_rating: 5 of 5 bubbles
service_rating: 5 of 5 bubbles
value_rating: 5 of 5 bubbles
atmosphere_rating: 5 of 5 bubbles
price_range: $20 -$100
feature: Free Wifi
mon: 6:00 pm - 11:30 pm
tue: 6:00 pm - 11:30 pm
wed: 6:00 pm - 11:30 pm
thu: 6:00 pm - 11:30 pm
fri: 6:00 pm - 11:30 pm
sat: 6:00 pm - 11:30 pm
sun: 6:00 pm - 11:30 pm
address: SOMA Neighborhood,San Francisco, CA 94107
description: Address:SOMA Neighborhood,San Francisco, CA 94107Location:United States&nbsp;&gt;&nbsp; California (CA)&nbsp;&gt;&nbsp; San FranciscoPhone Number:14154951900trkP:waypoint_for_poiPLACEMENT waypoint_for_poietk

----------------------------------------------------------------
title: Kokkari Estiatorio
popularity: #2 of 4,344 Restaurants in San Francisco
rating: 4

Raw txt file containing resturant information crawled from yelp (4 files in total, contains resturants of different prices)

In [6]:
!head -30 raw_data/yelp_crawling_price_1.txt

----------------------------------------------------------------
title: The Temporarium Coffee &amp; Tea
yelpPage: https://www.yelp.com/ /biz/the-temporarium-coffee-and-tea-san-francisco
categories: Coffee & Tea
price: Inexpensive
rating: 5.0 star rating
mon: 7:00 am-3:00 pm
tue: 7:00 am-3:00 pm
wed: 7:00 am-3:00 pm
thu: 7:00 am-3:00 pm
fri: 7:00 am-3:00 pm
sat: 7:00 am-6:30 pm
sun: 8:00 am-6:30 pm
addr: 3414 22nd StSan Francisco, CA 94111
phone: (415) 547-0616
takeout: Yes
creditCard: Yes
applePay: Yes
parking: Street
bikeParking: Yes
outdoorSeating: Yes
wifi: No
caters: No


----------------------------------------------------------------
title: Little Heaven Deli
yelpPage: https://www.yelp.com/ /biz/little-heaven-deli-san-francisco
categories: Creperies,Delis,Bubble Tea
price: Under $10


## Data table pre-processing

### Run parser code to extract attributes from the txt files and write into csv file 

In [7]:
!ls script

blocking.py        output.csv         schema_matching.py your_file.csv
file.txt           parse.ipynb        test.txt           your_file.xls


In [8]:
os.chdir('./script/')
!python schema_matching.py
os.chdir('..')

number of entries from tripadvisor_crawling_result: 3000
number of entries from yelp_crawling_price: 3010


Combine four seperate csv files of table B (yelp) into one table B csv file together

In [9]:
# this code block concatenate four yelps csv files in to one yelp file 
yelp1 = pd.read_csv('./managed_data/yelp_crawling_price_1.csv',dtype='str')
yelp2 = pd.read_csv('./managed_data/yelp_crawling_price_2.csv',dtype='str')
yelp3 = pd.read_csv('./managed_data/yelp_crawling_price_3.csv',dtype='str')
yelp4 = pd.read_csv('./managed_data/yelp_crawling_price_4.csv',dtype='str')
yelp5 = pd.read_csv('./managed_data/yelp_crawling_price_5.csv',dtype='str')
yelp_all = pd.concat([yelp1, yelp2, yelp3, yelp4, yelp5],axis=0)
yelp_all.to_csv('./managed_data/yelp_crawling_price_all.csv', index=False)

Data table Pre-treatment: 
* Read the table A and table B files 
* Convert all strings to lowercase for better matching
* Rewrite the converted tables to csv files

In [10]:
# Load csv files as dataframes and set the key attribute in the dataframe
Araw = pd.read_csv('./managed_data/tripadvisor_crawling_result.csv', dtype='str')
Braw = pd.read_csv('./managed_data/yelp_crawling_price_all.csv', dtype='str')

for t in list(Araw):
    Araw[t] = Araw[t].str.lower()
    
for t in list(Braw):
    Braw[t] = Braw[t].str.lower()
Araw.to_csv('./managed_data/tripadvisor_lowercase.csv', index=False)    
Braw.to_csv('./managed_data/yelp_lowercase.csv', index=False) 

## Read Table A and Table B and count the number of tuples 

In [11]:
path_A = './managed_data/tripadvisor_lowercase.csv'
path_B = './managed_data/yelp_lowercase.csv'
A = em.read_csv_metadata(path_A, key='id', dtype='str')
B = em.read_csv_metadata(path_B, key='id', dtype='str')

print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 3000
Number of tuples in B: 3010
Number of tuples in A X B (i.e the cartesian product): 9030000


In [12]:
A.head(3)

,id,title,cuisine,phone,feature,zipcode,street_addr
0,0,don of the bimini twist,"japanese,american,seafood",4154951900,free wifi,94107,soma neighborhood
1,1,kokkari estiatorio,"mediterranean,greek",4159810983,"reservations, private dining, seating, waitstaff, valet parking, wheelchair accessible, serves ...",94111,200 jackson street
2,2,molinari delicatessen,"italian,delicatessen",4154212337,"takeout, outdoor seating, wheelchair accessible",94133,373 columbus avenue north beach


In [13]:
B.head(3)

,id,title,cuisine,phone,feature,zipcode,street_addr
0,0,the temporarium coffee &amp; tea,coffee & tea,4155470616,"takeout, creditcard, applepay, bikeparking, outdoorseating",94111,3414 22nd street
1,1,little heaven deli,"creperies,delis,bubble tea",4158241293,"delivery, takeout, creditcard, bikeparking, goodforkids, waiterservice",94110,2348 mission street
2,2,zo11 coffee,"coffee & tea,sandwiches,coffee roasteries",4158729880,"takeout, creditcard, applepay, bikeparking",94115,1035 fillmore street


## Blocking candidate pairing set

* Use five strick blocking methods

    1. overlap blocker on title with overlap size =3
    
    2. attribute equivalence blocker on title
    
    3. overlap blocker on address with overlap size =3

    4. attribute equivalence blocker on address
    
    5. attribute equivalence blocker on phone number
    

* Union the result sets togehter

In [14]:
# Attribute Equivalence Blocking 
ab = em.AttrEquivalenceBlocker()
# Overlap Blocking
ob = em.OverlapBlocker()

# Block by Title
# when title size larger than 2, we need overlap=3 at least
CTitle_word_level = ob.block_tables(A, B, 'title', 'title',
                    l_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],
                    r_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],  
                      word_level=True, overlap_size=3, allow_missing=False, n_jobs=4)
# when title size is less than 3, we want exact match
CTitle_equivalence = ab.block_tables(A, B, 'title', 'title',
                    l_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],
                    r_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],  
                      allow_missing=False, n_jobs=4 )

# Block by Address
# when address size larger than 2, we need overlap=3 at least
CAddr_word_level = ob.block_tables(A, B, 'street_addr', 'street_addr',
                    l_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],
                    r_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],  
                      word_level=True, overlap_size=3, allow_missing=False, n_jobs=4)
# when address size is less than 3, we want exact match
CAddr_equivalence = ab.block_tables(A, B, 'street_addr', 'street_addr',
                    l_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],
                    r_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'], 
                      allow_missing=False, n_jobs=4 )


# Blocking based on phone number
CPhone = ab.block_tables(A, B, 'phone', 'phone',
                    l_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],
                    r_output_attrs=['title', 'cuisine', 'phone', 'feature', 'zipcode', 'street_addr'],
                    allow_missing=False,n_jobs=4)

# Combine blocker outputs
C = em.combine_blocker_outputs_via_union([CTitle_word_level, CTitle_equivalence, 
                                          CAddr_word_level, CAddr_equivalence, CPhone])
print('Number of tuples in CTitle_word_level: ' + str(len(CTitle_word_level)))
print('Number of tuples in CTitle_equivalence: ' + str(len(CTitle_equivalence)))
print('Number of tuples in CAddr_word_level: ' + str(len(CAddr_word_level)))
print('Number of tuples in CAddr_equivalence: ' + str(len(CAddr_equivalence)))
print('Number of tuples in CPhone: ' + str(len(CPhone)))
print('Number of tuples in C: ' + str(len(C)))

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


Number of tuples in CTitle_word_level: 442
Number of tuples in CTitle_equivalence: 704
Number of tuples in CAddr_word_level: 3665
Number of tuples in CAddr_equivalence: 1313
Number of tuples in CPhone: 948
Number of tuples in C: 3984


We tried several blocking approaches and decided to use the above approach.

This blocking approach can significantly reduce candidate set size and don't leave out any possible matching pairs.

From the debug_blocker tool, we can see the top tuples are not real matches, because of mismatch in address or restraunt names.


In [15]:
# Debug
dbg = em.debug_blocker(C, A, B, output_size=100)
dbg.head(20)

,_id,similarity,ltable_id,rtable_id,ltable_title,ltable_cuisine,ltable_phone,ltable_feature,ltable_zipcode,ltable_street_addr,rtable_title,rtable_cuisine,rtable_phone,rtable_feature,rtable_zipcode,rtable_street_addr
0,0,0.444444,1791,258,sutter cafe,NaN,4154343309,NaN,94108,330 sutter street,sutter st. cafe,"cafes,delis",4153628342,"takeout, creditcard, bikeparking, goodforkids, waiterservice, caters",94108,450 sutter street
1,1,0.444444,2297,569,cafe cole,cafe,4156687771,seating,94117,609 cole street,cole valley cafe,"coffee & tea,sandwiches,bagels",4156685282,"takeout, creditcard, bikeparking, goodforkids",94117,701 cole street
2,2,0.444444,1918,78,la taqueria menudo,mexican,4153330604,NaN,94112,4591 mission street,la taqueria,mexican,4152857117,"takeout, bikeparking, goodforkids, goodforgroups, outdoorseating",94110,2889 mission street
3,3,0.444444,930,78,la corneta taqueria,"mexican,latin,spanish",4156437001,"seating, wheelchair accessible",94110,2731 mission street,la taqueria,mexican,4152857117,"takeout, bikeparking, goodforkids, goodforgroups, outdoorseating",94110,2889 mission street
4,4,0.444444,2733,1268,yuubi japanese restaurant,japanese,4153862011,"delivery, takeout, reservations, seating, waitstaff, serves alcohol, wine and beer, accepts ame...",94118,501 balboa street,tekka japanese restaurant,"japanese,sushi bars",NaN,"hastv, waiterservice",94118,537 balboa street
5,5,0.444444,23,314,la taqueria,"mexican,latin,fast food, central american,south american",4152857117,"takeout, outdoor seating, seating, wheelchair accessible, serves alcohol",94110,2889 mission street,la corneta taqueria,mexican,4156437001,"reservations, takeout, creditcard, bikeparking, goodforkids, goodforgroups, hastv, caters",94110,2731 mission street
6,6,0.428571,2725,188,rumors sf,bar,NaN,NaN,94103,398 7th street,sf eagle,gay bars,NaN,"bikeparking, goodforgroups, outdoorseating, hastv",94103,398 12th street
7,7,0.416667,984,957,pera - san francisco,"mediterranean,turkish,greek",4157963812,"reservations, seating, waitstaff, serves alcohol",94107,1457 18th street,pizzeria delfina - san francisco,"pizza,italian",4154376800,"takeout, creditcard, bikeparking, goodforkids, outdoorseating, waiterservice",94110,3621 18th street
8,8,0.400000,1128,788,manzoni,italian,4153342251,"reservations, seating, waitstaff, wheelchair accessible",94131,2788 diamond street glen park,glen park station,"dive bars,sports bars,lounges",4153334633,"bikeparking, goodforgroups, hastv",94131,2816 diamond street
9,9,0.400000,680,2725,san tung chinese restaurant,"chinese,asian",4156614233,"takeout, seating, waitstaff, serves alcohol",94122,2240 irving street,san tung,"chinese,korean,asian fusion",4152420828,"takeout, creditcard, bikeparking, goodforkids, goodforgroups, waiterservice",94122,1031 irving street


## Train and Test Learning-based entity-matching

We downsampled 500 tuples from blocked set C and labeled it manually.

We selected 150 tuples first to see the density of matching tuples. The initial down sampled S set contains very few matching, about 5 in 150. Then we went back to modify the blocking approaches and did the down sampling again. After several iterations, we finally get the sampled S with reasonable high density.

The final blocking approach is shown above. We sampled 500 tuples, labeled them and saved the labeled data into csv file. So we can simply load the saved labeled data to do the learning-based entity-matching.

In [16]:
S = em.sample_table(C, 500)

In [34]:
C.to_csv('All_candidate.csv', index=False)

For simplicity, we directly load our previously labeled csv file

In [17]:
import numpy as np
path_gold = './managed_data/labeled_all.csv'
G = em.read_csv_metadata(path_gold, key='_id', ltable=A, rtable=B, 
                          fk_ltable='ltable_id',fk_rtable='rtable_id',dtype='str')

In [36]:
M

,_id,ltable_id,rtable_id,ltable_title,ltable_cuisine,ltable_phone,ltable_feature,ltable_zipcode,ltable_street_addr,rtable_title,rtable_cuisine,rtable_phone,rtable_feature,rtable_zipcode,rtable_street_addr,gold
0,0,0,2285,don of the bimini twist,"japanese,american,seafood",4154951900,free wifi,94107,soma neighborhood,the bimini twist,"personal chefs,american (new)",4154951900,"reservations, creditcard, bikeparking, goodforgroups, waiterservice, caters",94107,188 king street ph7,0
1,1,1,1267,kokkari estiatorio,"mediterranean,greek",4159810983,"reservations, private dining, seating, waitstaff, valet parking, wheelchair accessible, serves ...",94111,200 jackson street,kokkari estiatorio,"greek,mediterranean",4159810983,"reservations, creditcard, bikeparking, goodforgroups, waiterservice",94111,200 jackson street,0
2,2,10,1105,taylor street coffee shop,"american, contemporary,cafe",4155674031,"takeout, seating, waitstaff",94102,375 taylor street,taylor street coffee shop,"breakfast & brunch,burgers,sandwiches",4155674031,"takeout, creditcard, bikeparking, goodforkids, waiterservice",94102,375 taylor street,0
3,3,10,2734,taylor street coffee shop,"american, contemporary,cafe",4155674031,"takeout, seating, waitstaff",94102,375 taylor street,taylor street coffee shop,"breakfast & brunch,burgers,sandwiches",4155674031,"takeout, creditcard, bikeparking, goodforkids, waiterservice",94102,375 taylor street,0
4,4,10,2902,taylor street coffee shop,"american, contemporary,cafe",4155674031,"takeout, seating, waitstaff",94102,375 taylor street,taylor street coffee shop,"breakfast & brunch,burgers,sandwiches",4155674031,"takeout, creditcard, bikeparking, goodforkids, waiterservice",94102,375 taylor street,0
5,5,100,2259,quince,NaN,4157758500,"waitstaff, parking available, valet parking, wheelchair accessible, serves alcohol, full bar, a...",94133,470 pacific avenue,quince,"french,italian",4157758500,"reservations, creditcard, goodforgroups, waiterservice",94133,470 pacific avenue,0
6,6,1000,1308,piqueos,"peruvian,latin,spanish",4152828812,"reservations, seating, waitstaff, serves alcohol",94110,830 cortland,piqueo‰Ûªs,"latin american,peruvian",4152828812,"reservations, takeout, creditcard, bikeparking, goodforgroups, outdoorseating, waiterservice",94110,830 cortland avenue,0
7,7,1001,1820,lobster me,"american,seafood,fast food",NaN,"takeout, seating, wheelchair accessible",94103,865 market street in westfield,spa nordstrom,"day spas,massage,cosmetics & beauty supply",4159775102,creditcard,94103,865 market street,0
8,8,1001,1989,lobster me,"american,seafood,fast food",NaN,"takeout, seating, wheelchair accessible",94103,865 market street in westfield,club monaco san francisco - women‰Ûªs,women's clothing,4158560828,"creditcard, bikeparking",94108,865 market street,0
9,9,1001,2616,lobster me,"american,seafood,fast food",NaN,"takeout, seating, wheelchair accessible",94103,865 market street in westfield,soho,accessories,4155431021,creditcard,94105,865 market street,0


In [46]:
M = em.read_csv_metadata('./All_candidate.csv', key='_id', ltable=A, rtable=B, 
                          fk_ltable='ltable_id',fk_rtable='rtable_id',dtype='str')
M['ltable_zipcode'] = M['ltable_zipcode'].map(lambda x: np.random.randint(5000) if pd.isnull(x) else int(x))
M['rtable_zipcode'] = M['rtable_zipcode'].map(lambda x: np.random.randint(5000) if pd.isnull(x) else int(x))
M['ltable_phone'] = M['ltable_phone'].map(lambda x: np.random.randint(500000) if pd.isnull(x) else int(x))
M['rtable_phone'] = M['rtable_phone'].map(lambda x: np.random.randint(500000) if pd.isnull(x) else int(x))

for t in list(M):
    M[t] = M[t].map(lambda x: 'na' if pd.isnull(x) else x)
        
M['gold'] = M['gold'].map(lambda x: 0 if x=='0' else 1)

In [47]:
print('number of tuples in M :{}'.format(M.shape[0]))
print('number of matching tuples in M: {}'.format(np.sum(M['gold']==1)))
print('number of matching tuples in M: {}'.format(np.sum(M['gold']==0)))

number of tuples in M :3984
number of matching tuples in M: 0
number of matching tuples in M: 3984


Fill empty zipcode and phone number values with random number, because we don't want two null values have same values. 

In [18]:
G['ltable_zipcode'] = G['ltable_zipcode'].map(lambda x: np.random.randint(5000) if pd.isnull(x) else int(x))
G['rtable_zipcode'] = G['rtable_zipcode'].map(lambda x: np.random.randint(5000) if pd.isnull(x) else int(x))
G['ltable_phone'] = G['ltable_phone'].map(lambda x: np.random.randint(500000) if pd.isnull(x) else int(x))
G['rtable_phone'] = G['rtable_phone'].map(lambda x: np.random.randint(500000) if pd.isnull(x) else int(x))

for t in list(G):
    G[t] = G[t].map(lambda x: 'na' if pd.isnull(x) else x)
        
G['gold'] = G['gold'].map(lambda x: 0 if x=='0' else 1)

In [19]:
print('number of tuples in G :{}'.format(G.shape[0]))
print('number of matching tuples in G: {}'.format(np.sum(G['gold']==1)))
print('number of matching tuples in G: {}'.format(np.sum(G['gold']==0)))

number of tuples in G :486
number of matching tuples in G: 136
number of matching tuples in G: 350


### Testing a set of ML-matchers

In [20]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

### Use auto-generated feature vectors

In [21]:
# Generate features
feature_table = em.get_features_for_matching(A, B)
feature_table['feature_name']

0                                  id_id_lev_dist
1                                   id_id_lev_sim
2                                       id_id_jar
3                                       id_id_jwn
4                                       id_id_exm
5                           id_id_jac_qgm_3_qgm_3
6                     title_title_jac_qgm_3_qgm_3
7                 title_title_cos_dlm_dc0_dlm_dc0
8                 title_title_jac_dlm_dc0_dlm_dc0
9                                 title_title_mel
10                           title_title_lev_dist
11                            title_title_lev_sim
12                                title_title_nmw
13                                 title_title_sw
14                cuisine_cuisine_jac_qgm_3_qgm_3
15            cuisine_cuisine_cos_dlm_dc0_dlm_dc0
16            cuisine_cuisine_jac_dlm_dc0_dlm_dc0
17                            cuisine_cuisine_mel
18                       cuisine_cuisine_lev_dist
19                        cuisine_cuisine_lev_sim


### Maunually add more features

In [22]:
match_t = em.get_tokenizers_for_matching()
match_s = em.get_sim_funs_for_matching()
match_f = em.get_features_for_matching(A, B)
r1 = em.get_feature_fn('overlap_coeff(qgm_3(ltuple.title), qgm_3(rtuple.title))', match_t, match_s)
r2 = em.get_feature_fn('overlap_coeff(qgm_3(ltuple.street_addr), qgm_3(rtuple.street_addr))', match_t, match_s)


In [23]:
em.add_feature(feature_table, 'title_title_overlap_qgm3_qgm3', r1)
em.add_feature(feature_table, 'street_addr_street_addr_overlap_qgm3_qgm3', r1)
feature_table['feature_name']

0                                  id_id_lev_dist
1                                   id_id_lev_sim
2                                       id_id_jar
3                                       id_id_jwn
4                                       id_id_exm
5                           id_id_jac_qgm_3_qgm_3
6                     title_title_jac_qgm_3_qgm_3
7                 title_title_cos_dlm_dc0_dlm_dc0
8                 title_title_jac_dlm_dc0_dlm_dc0
9                                 title_title_mel
10                           title_title_lev_dist
11                            title_title_lev_sim
12                                title_title_nmw
13                                 title_title_sw
14                cuisine_cuisine_jac_qgm_3_qgm_3
15            cuisine_cuisine_cos_dlm_dc0_dlm_dc0
16            cuisine_cuisine_jac_dlm_dc0_dlm_dc0
17                            cuisine_cuisine_mel
18                       cuisine_cuisine_lev_dist
19                        cuisine_cuisine_lev_sim


In [48]:
GT = em.extract_feature_vecs(G, 
                            feature_table=feature_table, 
                            attrs_after=['gold'],
                            show_progress=False)
GT.fillna(0, inplace=True)
GT.head(5)

,_id,ltable_id,rtable_id,id_id_lev_dist,id_id_lev_sim,id_id_jar,id_id_jwn,id_id_exm,id_id_jac_qgm_3_qgm_3,title_title_jac_qgm_3_qgm_3,...,street_addr_street_addr_cos_dlm_dc0_dlm_dc0,street_addr_street_addr_jac_dlm_dc0_dlm_dc0,street_addr_street_addr_mel,street_addr_street_addr_lev_dist,street_addr_street_addr_lev_sim,street_addr_street_addr_nmw,street_addr_street_addr_sw,title_title_overlap_qgm3_qgm3,street_addr_street_addr_overlap_qgm3_qgm3,gold
0,312,1013,458,4,0.00,0.000000,0.000000,0,0.0,0.222222,...,0.218218,0.111111,0.521465,31.0,0.295455,-13.0,9.0,0.571429,0.571429,0
1,423,1020,507,3,0.25,0.527778,0.527778,0,0.0,0.238095,...,0.000000,0.000000,0.524405,17.0,0.150000,-1.0,3.0,0.416667,0.416667,0
2,868,1104,2652,4,0.00,0.000000,0.000000,0,0.0,0.108696,...,0.000000,0.000000,0.489815,18.0,0.100000,0.0,4.0,0.238095,0.238095,0
3,902,1104,810,2,0.50,0.722222,0.722222,0,0.0,0.195122,...,0.000000,0.000000,0.536700,19.0,0.136364,-1.0,4.0,0.421053,0.421053,0
4,910,1104,913,3,0.25,0.527778,0.527778,0,0.0,0.053571,...,0.000000,0.000000,0.451852,17.0,0.055556,1.0,3.0,0.103448,0.103448,0


In [49]:
MT = em.extract_feature_vecs(M, 
                            feature_table=feature_table, 
                            attrs_after=['gold'],
                            show_progress=False)
MT.fillna(0, inplace=True)
MT.head(5)

,_id,ltable_id,rtable_id,id_id_lev_dist,id_id_lev_sim,id_id_jar,id_id_jwn,id_id_exm,id_id_jac_qgm_3_qgm_3,title_title_jac_qgm_3_qgm_3,...,street_addr_street_addr_cos_dlm_dc0_dlm_dc0,street_addr_street_addr_jac_dlm_dc0_dlm_dc0,street_addr_street_addr_mel,street_addr_street_addr_lev_dist,street_addr_street_addr_lev_sim,street_addr_street_addr_nmw,street_addr_street_addr_sw,title_title_overlap_qgm3_qgm3,street_addr_street_addr_overlap_qgm3_qgm3,gold
0,0,0,2285,4.0,0.00,0.000000,0.000000,0,0.000000,0.592593,...,0.0,0.0,0.450538,17.0,0.105263,-1.0,3.0,0.888889,0.888889,0
1,1,1,1267,3.0,0.25,0.750000,0.775000,0,0.125000,1.000000,...,1.0,1.0,1.000000,0.0,1.000000,18.0,18.0,1.000000,1.000000,0
2,2,10,1105,2.0,0.50,0.833333,0.850000,0,0.111111,1.000000,...,1.0,1.0,1.000000,0.0,1.000000,17.0,17.0,1.000000,1.000000,0
3,3,10,2734,4.0,0.00,0.000000,0.000000,0,0.000000,1.000000,...,1.0,1.0,1.000000,0.0,1.000000,17.0,17.0,1.000000,1.000000,0
4,4,10,2902,3.0,0.25,0.583333,0.583333,0,0.000000,1.000000,...,1.0,1.0,1.000000,0.0,1.000000,17.0,17.0,1.000000,1.000000,0


In [50]:
matcher = lg
matcher.fit(table=GT, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')
# Predict on L 
predictions = matcher.predict(table=MT, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)

# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold', 'predicted')
em.print_eval_summary(eval_result)

Precision : 0.0% (0/1006)
Recall : 0.0% (0/0)
F1 : 0.0%
False positives : 1006 (out of 1006 positive predictions)
False negatives : 0 (out of 2978 negative predictions)


In [51]:
predictions.head()

,_id,ltable_id,rtable_id,id_id_lev_dist,id_id_lev_sim,id_id_jar,id_id_jwn,id_id_exm,id_id_jac_qgm_3_qgm_3,title_title_jac_qgm_3_qgm_3,...,street_addr_street_addr_jac_dlm_dc0_dlm_dc0,street_addr_street_addr_mel,street_addr_street_addr_lev_dist,street_addr_street_addr_lev_sim,street_addr_street_addr_nmw,street_addr_street_addr_sw,title_title_overlap_qgm3_qgm3,street_addr_street_addr_overlap_qgm3_qgm3,gold,predicted
0,0,0,2285,4.0,0.00,0.000000,0.000000,0,0.000000,0.592593,...,0.0,0.450538,17.0,0.105263,-1.0,3.0,0.888889,0.888889,0,0
1,1,1,1267,3.0,0.25,0.750000,0.775000,0,0.125000,1.000000,...,1.0,1.000000,0.0,1.000000,18.0,18.0,1.000000,1.000000,0,1
2,2,10,1105,2.0,0.50,0.833333,0.850000,0,0.111111,1.000000,...,1.0,1.000000,0.0,1.000000,17.0,17.0,1.000000,1.000000,0,1
3,3,10,2734,4.0,0.00,0.000000,0.000000,0,0.000000,1.000000,...,1.0,1.000000,0.0,1.000000,17.0,17.0,1.000000,1.000000,0,1
4,4,10,2902,3.0,0.25,0.583333,0.583333,0,0.000000,1.000000,...,1.0,1.000000,0.0,1.000000,17.0,17.0,1.000000,1.000000,0,1


In [65]:
MG = M.copy()
MG['gold'] = predictions['predicted']
MK = MG.ix[MG['gold']==1]
MK.head()
MK.to_csv('matched_data.csv', index=False)

### Split the dataset G into development set and testing set

In [24]:
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

In [25]:
# Convert the I into a set of feature vectors using F


H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after=['gold'],
                            show_progress=False)
H.fillna(0, inplace=True)
H.head(5)

,_id,ltable_id,rtable_id,id_id_lev_dist,id_id_lev_sim,id_id_jar,id_id_jwn,id_id_exm,id_id_jac_qgm_3_qgm_3,title_title_jac_qgm_3_qgm_3,...,street_addr_street_addr_cos_dlm_dc0_dlm_dc0,street_addr_street_addr_jac_dlm_dc0_dlm_dc0,street_addr_street_addr_mel,street_addr_street_addr_lev_dist,street_addr_street_addr_lev_sim,street_addr_street_addr_nmw,street_addr_street_addr_sw,title_title_overlap_qgm3_qgm3,street_addr_street_addr_overlap_qgm3_qgm3,gold
64,338,1254,2154,2,0.500000,0.916667,0.916667,0,0.200000,0.0,...,0.750000,0.6,0.946394,3.0,0.842105,15.0,16.0,0.0,0.0,0
301,945,186,124,2,0.333333,0.555556,0.600000,0,0.111111,1.0,...,0.333333,0.2,0.603704,10.0,0.444444,7.0,8.0,1.0,1.0,0
411,2049,2975,2707,3,0.250000,0.666667,0.700000,0,0.090909,0.0,...,1.000000,1.0,1.000000,0.0,1.000000,16.0,16.0,0.0,0.0,0
81,695,162,572,2,0.333333,0.555556,0.555556,0,0.111111,0.0,...,0.774597,0.6,0.914130,7.0,0.695652,9.0,12.0,0.0,0.0,0
347,1454,2439,130,3,0.250000,0.527778,0.527778,0,0.000000,1.0,...,1.000000,1.0,1.000000,0.0,1.000000,16.0,16.0,1.0,1.0,1


In [26]:
# Select the best ML matcher using CV by precision
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=5,
        target_attr='gold', metric='precision', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x1163f8668>,5,0.904762,0.900000,0.800000,0.928571,0.950000,0.896667
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x1163f85c0>,5,1.000000,1.000000,0.772727,1.000000,1.000000,0.954545
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x1163f8630>,5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x1163f81d0>,5,1.000000,0.954545,0.900000,1.000000,0.944444,0.959798
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x1163f85f8>,5,1.000000,0.954545,0.857143,0.933333,1.000000,0.949004
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x1163f8b38>,5,0.913043,1.000000,0.900000,0.923077,0.944444,0.936113


In [27]:
# Select the best ML matcher using CV by recall
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=5,
        target_attr='gold', metric='recall', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x1163f8668>,5,0.863636,0.857143,0.888889,0.928571,0.95,0.897648
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x1163f85c0>,5,0.909091,1.000000,0.944444,0.928571,0.85,0.926421
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x1163f8630>,5,0.318182,0.476190,0.666667,0.642857,0.50,0.520779
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x1163f81d0>,5,0.954545,1.000000,1.000000,1.000000,0.85,0.960909
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x1163f85f8>,5,0.863636,1.000000,1.000000,1.000000,0.85,0.942727
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x1163f8b38>,5,0.954545,1.000000,1.000000,0.857143,0.85,0.932338


In [28]:
# Select the best ML matcher using CV by f1 score
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=5,
        target_attr='gold', metric='f1', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x1163f8668>,5,0.883721,0.878049,0.842105,0.928571,0.950000,0.896489
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x1163f85c0>,5,0.952381,1.000000,0.850000,0.962963,0.918919,0.936853
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x1163f8630>,5,0.482759,0.645161,0.800000,0.782609,0.666667,0.675439
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x1163f81d0>,5,0.976744,0.976744,0.947368,1.000000,0.894737,0.959119
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x1163f85f8>,5,0.926829,0.976744,0.923077,0.965517,0.918919,0.942217
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x1163f8b38>,5,0.933333,1.000000,0.947368,0.888889,0.894737,0.932865


### Debugging matcher

To find if there is any labeling error or attribute extraction error.

Split the development set into two equal set U and V for training and testing, respectively.

In [29]:

UV = em.split_train_test(H, train_proportion=0.5,random_state=0)
U = UV['train']
V = UV['test']

In [30]:
em.vis_debug_rf(rf, U, V, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], target_attr='gold')

In [31]:
em.vis_debug_dt(dt, U, V, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], target_attr='gold')

**We fixed mislabeled tuples by examing the debugger, and find that the random forest and decision tree is weak in minimizing false negatives. Matching tuples with title or address overlaped are not detected. **

After debugging, we reached the satisfying precision and recall rate and chose Logisitc Regression as the best matchers. 

In [32]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, feature_table=feature_table,
                            attrs_after='gold', show_progress=False)
L.fillna(0, inplace=True)

In [33]:
matcher = lg
matcher.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')
# Predict on L 
predictions = matcher.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)

# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold', 'predicted')
em.print_eval_summary(eval_result)

Precision : 100.0% (38/38)
Recall : 92.68% (38/41)
F1 : 96.2%
False positives : 0 (out of 38 positive predictions)
False negatives : 3 (out of 108 negative predictions)


In [72]:
a = 'Indian,Gluten Free Options'
A = a.split(',')

In [73]:
b = 'Indian,Vegetarian'
B = b.split(',')

In [75]:
resulting_list = list(A)
resulting_list.extend(x for x in B if x not in resulting_list)
', '.join(resulting_list)

'Indian, Gluten Free Options, Vegetarian'

In [104]:
trip_price = '$30.4 -$35'
yelp_price = '$11-30'

In [105]:
prices_A = re.split('\$|-| -|\n', trip_price)
prices_B = re.split('\$|-| -|\n', yelp_price)
prices = prices_A + prices_B
nums = [s for s in prices if s.replace('.', '', 1).isdigit()]

In [106]:
nums

['30.4', '35', '11', '30']

In [86]:
nums = [s for s in rating.split() if s.replace('.','',1).isdigit()]
'/'.join(nums)

'4.0/5.0'

In [ ]:
new